In [117]:
import constants
import json
import os

In [118]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
openai_key = os.getenv("OPENAI_API_KEY")

In [119]:
response_schemas = [
    ResponseSchema(name="clarity_score", description="score of clarity of learner's explanation"),
    ResponseSchema(name="clarity_explanation", description="explanation of clarity score given"),
    ResponseSchema(name="conciseness_score", description="score of conciseness of learner's explanation"),
    ResponseSchema(name="conciseness_explanation", description="explanation of conciseness score given"),
    ResponseSchema(name="comprehensiveness_score", description="score of comprehensiveness of learner's explanation"),
    ResponseSchema(name="comprehensiveness_explanation", description="explanation of comprehensiveness score given"),
    ResponseSchema(name="correctness_score", description="score of correctness of learner's explanation"),
    ResponseSchema(name="correctness_explanation", description="explanation of correctness score given"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"clarity_score": string  // score of clarity of learner\'s explanation\n\t"clarity_explanation": string  // explanation of clarity score given\n\t"conciseness_score": string  // score of conciseness of learner\'s explanation\n\t"conciseness_explanation": string  // explanation of conciseness score given\n\t"comprehensiveness_score": string  // score of comprehensiveness of learner\'s explanation\n\t"comprehensiveness_explanation": string  // explanation of comprehensiveness score given\n\t"correctness_score": string  // score of correctness of learner\'s explanation\n\t"correctness_explanation": string  // explanation of correctness score given\n}\n```'

In [120]:
template = "You are a helpful assistant that takes a transcript and scores the learner's explanation to the agent and provides a detailed explanation using the evidence available. You will score using this rubric: {rubric} and you can use the agent's response to support your case but remember that you are scoring the learner's explanations. \n{format_instructions}\n{question}"

prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question", "rubric"],
    partial_variables={"format_instructions": format_instructions}
)
with open('./conversation_data.json', 'r') as file:
    conversation = json.load(file)
    _input = prompt.format_prompt(question=conversation, rubric=constants.marking_rubric)

chat_model = ChatOpenAI(model="gpt-4", openai_api_key=openai_key)
output = chat_model(_input.to_messages())
output_parser.parse(output.content)

{'clarity_score': '4.8',
 'clarity_explanation': 'The learner provided a clear and detailed explanation of merge sort. They effectively described the process of dividing the list, sorting the smaller lists, and merging them back together. However, there were a few instances where the learner could have used more precise language to describe the process.',
 'conciseness_score': '4.5',
 'conciseness_explanation': "The learner's explanation was mostly concise. However, they occasionally included extra information or repeated certain points, which slightly detracted from the conciseness of their explanation.",
 'comprehensiveness_score': '5',
 'comprehensiveness_explanation': "The learner's explanation covered all aspects of merge sort, including how it works, its time complexity, and how it compares to other sorting algorithms. They also discussed potential drawbacks of merge sort, such as its memory usage.",
 'correctness_score': '5',
 'correctness_explanation': 'The information provided